# Вакансии с сайта hh.ru

### Импортируем необходимые библиотеки

In [1]:
import re
import time

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

### Основные переменные и функции для получения ссылок на страницы вакансий

In [2]:
url = (
    "https://hh.ru/search/vacancy?order_by=relevance&items_on_page=50&L_save_area=true"
)
text = '"аналитик+данных"+OR+"data+analyst"+OR+"бизнес-аналитик"+OR+"ML+engineer"+OR+"data+engineer"+OR+"инженер+данных"+OR+"Data+science"+OR+ML'
salary = ""
label = "label=with_salary"
search_field = "search_field=name&search_field=description"
area = (
    "area=1002&area=1&area=2&area=159&area=160&area=68&area=90&area=4&area=54&area=102"
)
experience = "experience=doesNotMatter"
hhtmFrom = "vacancy_search_filter"
hhtmFromLabel = ""

all_vacancies = []


def get_page_soup(page_source):
    return BeautifulSoup(page_source, "html.parser")


def get_vacancies_urls(soup):
    links = []
    for link in soup.findAll("a", {"data-qa": "serp-item__title"}):
        # исключаем рекламные вакансии
        if link["href"].count("adsrv"):
            continue
        links.append(link["href"])
    return links


def collect_vacancies_urls_list(br, max_links=1000):
    vacancies_urls_result = []

    wait = WebDriverWait(br, 10)

    while len(vacancies_urls_result) < max_links:
        time.sleep(1)

        # Получаем HTML и парсим ссылки
        soup = get_page_soup(br.page_source)
        links = get_vacancies_urls(soup)

        if links:
            vacancies_urls_result.extend(links)
            print(f"Собрано {len(vacancies_urls_result)} ссылок...")
        else:
            print("На странице нет ссылок. Остановка.")
            break

        # Принять cookies, если есть на странице
        try:
            policy_button = wait.until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "#bottom-cookies-policy-informer button")
                )
            )
            policy_button.click()
            time.sleep(0.5)
        except:
            pass

        # Пытаемся найти кнопку следующей страницы
        try:
            next_page_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-qa="pager-next"]'))
            )
        except:
            print("Кнопка следующей страницы не найдена. Остановка.")
            break

        # Переходим на следующую страницу
        next_page_button.click()

        # Ждём прогрузку новой страницы
        time.sleep(2)

    return vacancies_urls_result

### Запуск Вебдрайвера на главной странице поиска вакансий и сбор ссылок на вакансии по указанным параметрам

In [3]:
# Запуск. Собираем ссылки на вакансии
br = webdriver.Chrome()
full_url = f'{url}&text={text}&salary={salary}&{area}&{search_field}&{label}&hhtmFrom={hhtmFrom}&hhtmFromLabel={hhtmFromLabel}'
br.get(full_url)
# Для управления количеством вакансий можно передать параметр max_links кратный 50, так как за один проход на странице собираем 50 ссылок
# Если параметр не передан - используется дефолтное значение max_links=1000
all_vacancies = collect_vacancies_urls_list(br, max_links=10)
# Закрываем браузер
br.close()
print(f'\nИтого собрано ссылок: {len(all_vacancies)}')

The chromedriver version (142.0.7444.162) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (143.0.7499.41); currently, chromedriver 143.0.7499.42 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry
C:\Users\Егор\AppData\Local\Temp\ipykernel_24392\2530552583.py:24: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  for link in soup.findAll("a", {"data-qa": "serp-item__title"}):


Собрано 50 ссылок...

Итого собрано ссылок: 50


## Итоговый датасет. Описание признаков

```bash
'name': string # Название вакансии
'salary': int  # Зарплата за месяц
'currency': string  # Валюта зарплаты
'experience': string  # Требуемый опыт в годах: 0, 1-3, 3-6, 6
'location': string  # Город / Страна размещения вакансии:  'Москва / Россия'
'skills': string  # Требуемый Стэк, ключевые навыки: 'SQL, NoSQL, Python, Базы данных, Big Data'. Разделитель: - запятая, пробел
'link': string  # ссылка на вакансию
```


### Функции для обработки признаков Salary, Currency, Frequency

In [4]:
currency_map = {
    '₸': 'KZT',
    '₽': 'RUB',
    '€': 'EUR',
    '$': 'USD',
    'Br': 'BYN',
}


def split_salary(salary_list):
    salary = 0
    currency_code = '₽'
    if 'от' in salary_list:
        index = salary_list.index('от')
        salary = int(salary_list[index + 1].replace('\xa0', ''))
        if 'до' in salary_list:
            index = salary_list.index('до')
        currency_code = salary_list[index + 2]
    elif 'до' in salary_list:
        index = salary_list.index('до')
        salary = int(salary_list[index + 1].replace('\xa0', ''))
        currency_code = salary_list[index + 2]
    else:
        if salary_list[0].count(' '):
            salary = int(salary_list[0].replace('\xa0', ''))
            currency_code = salary_list[1]
        else:
            salary_currency_list = salary_list[0].split('\xa0')
            salary = int(salary_currency_list[0] + salary_currency_list[1])
            currency_code = salary_currency_list[2]

    frequency = str(salary_list[-1]).replace('\xa0', ' ')

    currency = currency_map.get(currency_code, currency_code)

    return (salary, currency, frequency)


def get_salary(soup):
    # span data-qa="vacancy-salary-compensation-type-gross" - до вычета налогов
    # span data-qa="vacancy-salary-compensation-type-net" - на руки
    salary = 0
    currency = ''
    frequency = ''

    all_salary_gross = soup.find(
        'span', {'data-qa': 'vacancy-salary-compensation-type-gross'}
    )
    all_salary_net = soup.find(
        'span', {'data-qa': 'vacancy-salary-compensation-type-net'}
    )
    if all_salary_gross:
        all_salary_gross_list = str(all_salary_gross.text).split(', ')[0].split(' ')
        salary, currency, frequency = split_salary(all_salary_gross_list)

    elif all_salary_net:
        all_salary_net_list = str(all_salary_net.text).split(', ')[0].split(' ')
        salary, currency, frequency = split_salary(all_salary_net_list)

    return (salary, currency, frequency)

### Функции для обработки признаков Location

In [5]:
city_map = {
    'Минске': 'Минск / Беларусь',
    'Санкт-Петербурге': 'Санкт-Петербург / Россия',
    'Москве': 'Москва / Россия',
    'Томске': 'Томск / Россия',
    'Красноярске': 'Красноярск / Россия',
    'Омске': 'Омск / Россия',
    'Хабаровске': 'Хабаровск / Россия',
    'Новосибирске': 'Новосибирск / Россия',
    'Алматы': 'Алматы / Казахстан',
    'Астане': 'Астана / Казахстан',
}


def extract_city_nom(raw_text):
    match = re.search(r"в\s+([А-ЯЁа-яё\- ]+)$", raw_text)
    if not match:
        return None

    city = match.group(1).strip()

    if city in city_map:
        return city_map[city]
    return city

### Функции для обработки признаков Experience

In [6]:
def get_expeience(text):
    if text == 'не требуется':
        return str(0)
    elif 'более' in text:
        _, exp, _ = text.split(' ')
        return exp
    elif 'лет' in text or 'года' in text:
        exp, _ = text.split(' ')
        return exp
    else:
        return text

### Основная функции для сбора всех необходимых признаков по вакансии

In [7]:
def collect_vacancy_data(br):
    vacancy_result = {}

    time.sleep(1)

    # Получаем HTML
    soup = get_page_soup(br.page_source)

    # Если вакансия в архиве - пропускаем ее
    is_archived = soup.find('div', {'data-qa': 'vacancy-title-archived-text'})
    if is_archived:
        return {}
    # найдем name
    title = soup.find('h1', {'data-qa': 'vacancy-title'})

    name = None
    if title:
        if title.find('span'):
            name = title.find('span').text
        else:
            name = title.text
    vacancy_result['name'] = name

    # найдем salary, currency, frequency
    salary, currency, frequency = get_salary(soup)
    vacancy_result['salary'] = salary
    vacancy_result['currency'] = currency
    vacancy_result['frequency'] = frequency

    # найдем experience
    # span data-qa="vacancy-experience"
    experience_el = soup.find('span', {'data-qa': 'vacancy-experience'})
    if experience_el:
        experience_text = experience_el.text
        vacancy_result['experience'] = get_expeience(experience_text)

    # найдем location
    # city_elem = soup.find(string=lambda s: "Вакансия опубликована" in s)
    city_elem = br.find_element(
        By.XPATH,
        "//div[contains(@class,'magritte-text_style-secondary') and contains(text(),'Вакансия опубликована')]",
    )
    raw_text = city_elem.text
    vacancy_result['location'] = extract_city_nom(raw_text)

    # skills
    # li data-qa="skills-element"
    skills_list = soup.findAll('li', {'data-qa': 'skills-element'})
    if skills_list:
        skills_result = []
        for skill in skills_list:
            skills_result.append(skill.text)
        vacancy_result['skills'] = ', '.join(skills_result)

    return vacancy_result

In [8]:
all_vacancies_result = []

# Запуск Сбор данных по вакансии
for vacancy in all_vacancies:

    full_url = str(vacancy).split('?')[0]
    br = webdriver.Chrome()
    br.get(full_url)
    vacancy_data = collect_vacancy_data(br)
    if not vacancy_data.keys():
        continue
    vacancy_data['link'] = full_url
    all_vacancies_result.append(vacancy_data)

    br.close()

The chromedriver version (142.0.7444.162) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (143.0.7499.41); currently, chromedriver 143.0.7499.42 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry
C:\Users\Егор\AppData\Local\Temp\ipykernel_24392\2806931398.py:48: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  skills_list = soup.findAll('li', {'data-qa': 'skills-element'})
The chromedriver version (142.0.7444.162) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (143.0.7499.41); currently, chromedriver 143.0.7499.42 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (142.0.7444.162) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (143.0.7499.41); currently, chromedriver 143.0.7499.42 is 

KeyboardInterrupt: 

### Создаем DataFrame

In [ ]:
df = pd.DataFrame(all_vacancies_result)
df.shape

### Сохранаяем только вакансии с зарплатой за месяц. Признак Frequency удалялем

In [ ]:
df['frequency'].value_counts()

In [ ]:
df_final_1 = df[df['frequency'] == 'за месяц']
df_final_2 = df_final_1.drop('frequency', axis=1)
df_final_2.shape


### Сохраняем итоговый датасет в файл .csv

In [ ]:
df_final_2.to_csv("hh_vacancies.csv", index=False)